In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import warnings
import psycopg2
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
import sys
import pickle
from pprint import pprint
from sklearn.preprocessing import LabelEncoder

sys.path.insert(0, os.path.realpath('..\..\src'))
from database.database_config import DB_NAME, DB_USER, DB_PASSWORD, DB_HOST

In [2]:
db = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
conn = db.connect()

In [3]:
SQL = """
select
	match_id,
	mf.date_id,
	mf.venue_id,
	mf.referee_id,
	home_team_id as home_id,
	away_team_id as away_id,
	attendance,
    match_week,
	xg_home,
	xg_away,
	home_goal,
	away_goal,
	home_captain_name,
	away_captain_name,
	home_formation,
	away_formation,
	home_possession,
	away_possession,
	home_shots,
	home_shots_on_target,
	home_distance,
	home_freekicks,
	home_penalty_kicks,
	home_touches,
	home_touches_att_pen,
	home_touches_def_pen,
	home_touches_def_third,
	home_touches_mid_third,
	home_touches_att_third,
	home_touches_live,
	home_dribbles_success,
	home_dribbles_att,
	home_dribbles_players_number,
	home_dribbles_megs,
	home_carries,
	home_carries_tot_dist,
	home_carries_prog_dist,
	home_carries_prog,
	home_carries_one_third,
	home_carries_cpa,
	home_carries_miss,
	home_carries_dis,
	home_receiving_target,
	home_receiving_rec,
	home_receiving_prog,
	away_shots,
	away_shots_on_target,
	away_distance,
	away_freekicks,
	away_penalty_kicks,
	away_touches,
	away_touches_att_pen,
	away_touches_def_pen,
	away_touches_def_third,
	away_touches_mid_third,
	away_touches_att_third,
	away_touches_live,
	away_dribbles_success,
	away_dribbles_att,
	away_dribbles_players_number,
	away_dribbles_megs,
	away_carries,
	away_carries_tot_dist,
	away_carries_prog_dist,
	away_carries_prog,
	away_carries_one_third,
	away_carries_cpa,
	away_carries_miss,
	away_carries_dis,
	away_receiving_target,
	away_receiving_rec,
	away_receiving_prog,
	match_result,
	ht.team_name as home_team,
	aw.team_name as away_team,
	md."_date_",
	mr.referee_name,
	mv.venue_name
from
	laliga_wh.match_facts mf
inner join laliga_wh.match_teams ht 
on
	ht.team_id = mf.home_team_id
inner join laliga_wh.match_teams aw 
on
	aw.team_id = mf.away_team_id
inner join laliga_wh.match_dates md 
on
	md.date_id = mf.date_id
inner join laliga_wh.match_referees mr 
on
	mr.referee_id = mf.referee_id
inner join laliga_wh.match_venues mv 
on
	mv.venue_id = mf.venue_id
"""


In [4]:
match_df = pd.read_sql(SQL, con=conn)

In [5]:
match_df

,match_id,date_id,venue_id,referee_id,home_id,away_id,attendance,match_week,xg_home,xg_away,...,away_carries_dis,away_receiving_target,away_receiving_rec,away_receiving_prog,match_result,home_team,away_team,_date_,referee_name,venue_name
0,1,1,13,22,17,1,9231.0,1,1.4,1.2,...,15,309,250,250,HW,Leganés,Alavés,2017-08-18,José Munuera,Estadio Municipal de Butarque
1,2,2,25,18,26,16,35971.0,1,1.9,0.3,...,12,499,451,451,HW,Valencia,Las Palmas,2017-08-18,Jesús Gil,Estadio de Mestalla
2,3,3,22,7,6,24,16961.0,1,1.6,2.4,...,7,419,352,352,AW,Celta Vigo,Real Sociedad,2017-08-19,Antonio Matéu Lahoz,Estadio de Balaídos
3,4,4,3,24,12,3,11511.0,1,2.1,0.8,...,8,406,347,347,D,Girona,Atlético Madrid,2017-08-19,Juan Martínez,Estadi Municipal de Montilivi
4,5,5,18,4,25,10,30487.0,1,2.1,1.1,...,12,401,297,297,D,Sevilla,Espanyol,2017-08-19,Alejandro Hernández,Estadio Ramón Sánchez Pizjuán
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,1896,1732,16,4,13,10,17951.0,38,2.0,0.4,...,13,487,466,466,D,Granada,Espanyol,2022-05-22,Alejandro Hernández,Estadio Nuevo Los Cármenes
1896,1897,1732,8,9,21,19,18717.0,38,0.8,1.3,...,5,270,211,211,AW,Osasuna,Mallorca,2022-05-22,César Soto,Estadio El Sadar
1897,1898,1733,1,21,4,28,54850.0,38,0.7,0.7,...,5,300,254,254,AW,Barcelona,Villarreal,2022-05-22,José Luis Munuera,Camp Nou
1898,1899,1733,12,18,24,3,23586.0,38,2.3,0.8,...,11,363,281,281,AW,Real Sociedad,Atlético Madrid,2022-05-22,Jesús Gil,Estadio Municipal de Anoeta


In [6]:
match_df.dtypes

match_id         int64
date_id          int64
venue_id         int64
referee_id       int64
home_id          int64
                 ...  
home_team       object
away_team       object
_date_          object
referee_name    object
venue_name      object
Length: 78, dtype: object

# Preprocess
We will drop the last few columns pulled in from the joins
We have to verify data types are correct. Match to what is in the data warehouse
Clean formation columns 

In [7]:
match_results = match_df.iloc[:,:73]

In [8]:
match_results.head()

,match_id,date_id,venue_id,referee_id,home_id,away_id,attendance,match_week,xg_home,xg_away,...,away_carries_prog_dist,away_carries_prog,away_carries_one_third,away_carries_cpa,away_carries_miss,away_carries_dis,away_receiving_target,away_receiving_rec,away_receiving_prog,match_result
0,1,1,13,22,17,1,9231.0,1,1.4,1.2,...,758,21.0,5.0,1.0,15,15,309,250,250,HW
1,2,2,25,18,26,16,35971.0,1,1.9,0.3,...,1439,52.0,15.0,2.0,15,12,499,451,451,HW
2,3,3,22,7,6,24,16961.0,1,1.6,2.4,...,1013,30.0,10.0,3.0,14,7,419,352,352,AW
3,4,4,3,24,12,3,11511.0,1,2.1,0.8,...,1169,44.0,15.0,3.0,5,8,406,347,347,D
4,5,5,18,4,25,10,30487.0,1,2.1,1.1,...,755,32.0,17.0,3.0,5,12,401,297,297,D


In [9]:
for i,v in match_results.dtypes.iteritems():
    print(i, v)

match_id int64
date_id int64
venue_id int64
referee_id int64
home_id int64
away_id int64
attendance float64
match_week int64
xg_home float64
xg_away float64
home_goal int64
away_goal int64
home_captain_name object
away_captain_name object
home_formation object
away_formation object
home_possession float64
away_possession float64
home_shots int64
home_shots_on_target int64
home_distance float64
home_freekicks int64
home_penalty_kicks int64
home_touches int64
home_touches_att_pen int64
home_touches_def_pen int64
home_touches_def_third int64
home_touches_mid_third int64
home_touches_att_third int64
home_touches_live int64
home_dribbles_success int64
home_dribbles_att int64
home_dribbles_players_number int64
home_dribbles_megs int64
home_carries int64
home_carries_tot_dist int64
home_carries_prog_dist int64
home_carries_prog float64
home_carries_one_third float64
home_carries_cpa float64
home_carries_miss int64
home_carries_dis int64
home_receiving_target int64
home_receiving_rec int64
hom

In [10]:
# Fix data types for certain columns
match_results['attendance'] = match_results['attendance'].astype(pd.Int64Dtype())

In [11]:
match_results['home_formation'].value_counts()

4-4-2        636
4-2-3-1      363
4-3-3        273
4-1-4-1      144
3-5-2        125
3-4-3         89
4-4-2◆        62
4-2-2-2       56
4-4-1-1       41
4-5-1         22
3-2-2-2-1     14
4-3-2-1       12
3-4-1-2       11
3-1-4-2        8
3-3-2-2        7
3-2-3-2        5
4-3-1-2◆       5
3-2-2-1-2      4
3-5-1-1        4
4-3-3◆         3
4-2-2-1-1      3
5-3-2          2
5-4-1          2
4-1-3-2        1
4-1-3-2◆       1
4-3-1-2        1
3-1-4-1-1      1
4-2-3-1◆       1
4-1-2-3        1
3-2-2-3        1
3-2-1-2-2      1
3-4-3◆         1
Name: home_formation, dtype: int64

In [12]:
match_results['home_formation'] = match_results['home_formation'].replace({'4-4-2◆': '4-4-2', '4-3-1-2◆':'4-3-1-2', '4-3-3◆':'4-3-3', '4-1-3-2◆':'4-1-3-2',
                                         '4-2-3-1◆':'4-2-3-1', '3-4-3◆':'3-4-3'})
match_results['away_formation'] = match_results['away_formation'].replace({'4-4-2◆': '4-4-2', '4-3-1-2◆':'4-3-1-2', '4-3-3◆':'4-3-3', '4-1-3-2◆':'4-1-3-2',
                                         '4-2-3-1◆':'4-2-3-1', '3-4-3◆':'3-4-3', '4-3-2-1◆':'4-3-2-1'})

In [13]:
match_results['home_formation'].value_counts()

4-4-2        698
4-2-3-1      364
4-3-3        276
4-1-4-1      144
3-5-2        125
3-4-3         90
4-2-2-2       56
4-4-1-1       41
4-5-1         22
3-2-2-2-1     14
4-3-2-1       12
3-4-1-2       11
3-1-4-2        8
3-3-2-2        7
4-3-1-2        6
3-2-3-2        5
3-5-1-1        4
3-2-2-1-2      4
4-2-2-1-1      3
5-3-2          2
5-4-1          2
4-1-3-2        2
4-1-2-3        1
3-1-4-1-1      1
3-2-2-3        1
3-2-1-2-2      1
Name: home_formation, dtype: int64

In [14]:
match_results['away_formation'].value_counts()

4-4-2        642
4-2-3-1      380
4-3-3        255
4-1-4-1      151
3-5-2        145
3-4-3        107
4-2-2-2       59
4-4-1-1       36
4-3-2-1       21
4-5-1         18
3-2-2-2-1     15
3-1-4-2       13
3-4-1-2       11
3-3-2-2       11
3-2-3-2        8
3-5-1-1        7
5-3-2          6
4-3-1-2        4
5-4-1          3
3-1-2-2-2      2
4-1-3-2        2
3-2-2-1-2      2
5-1-2-2        2
Name: away_formation, dtype: int64

In [15]:
for i,v in match_results.dtypes.iteritems():
    print(i, v)

match_id int64
date_id int64
venue_id int64
referee_id int64
home_id int64
away_id int64
attendance Int64
match_week int64
xg_home float64
xg_away float64
home_goal int64
away_goal int64
home_captain_name object
away_captain_name object
home_formation object
away_formation object
home_possession float64
away_possession float64
home_shots int64
home_shots_on_target int64
home_distance float64
home_freekicks int64
home_penalty_kicks int64
home_touches int64
home_touches_att_pen int64
home_touches_def_pen int64
home_touches_def_third int64
home_touches_mid_third int64
home_touches_att_third int64
home_touches_live int64
home_dribbles_success int64
home_dribbles_att int64
home_dribbles_players_number int64
home_dribbles_megs int64
home_carries int64
home_carries_tot_dist int64
home_carries_prog_dist int64
home_carries_prog float64
home_carries_one_third float64
home_carries_cpa float64
home_carries_miss int64
home_carries_dis int64
home_receiving_target int64
home_receiving_rec int64
home_

In [16]:
# create numerical codes for certain columns 
le = LabelEncoder()
le.fit(match_results[['home_captain_name', 'away_captain_name']].stack().unique())
match_results['home_captain_id'] = le.transform(match_results['home_captain_name'])
match_results['away_captain_id'] = le.transform(match_results['away_captain_name'])
match_results['match_result_code'] = match_results['match_result'].replace({'HW' : 0, 'AW':1, 'D':2})

In [17]:
match_results.head()

,match_id,date_id,venue_id,referee_id,home_id,away_id,attendance,match_week,xg_home,xg_away,...,away_carries_cpa,away_carries_miss,away_carries_dis,away_receiving_target,away_receiving_rec,away_receiving_prog,match_result,home_captain_id,away_captain_id,match_result_code
0,1,1,13,22,17,1,9231,1,1.4,1.2,...,1.0,15,15,309,250,250,HW,131,121,0
1,2,2,25,18,26,16,35971,1,1.9,0.3,...,2.0,15,12,499,451,451,HW,39,99,0
2,3,3,22,7,6,24,16961,1,1.6,2.4,...,3.0,14,7,419,352,352,AW,71,190,1
3,4,4,3,24,12,3,11511,1,2.1,0.8,...,3.0,5,8,406,347,347,D,194,59,2
4,5,5,18,4,25,10,30487,1,2.1,1.1,...,3.0,5,12,401,297,297,D,93,64,2


In [18]:
match_results[['home_captain_name','away_captain_name', 'home_captain_id', 'away_captain_id']].loc[(match_results['away_captain_id'] == 131)]

,home_captain_name,away_captain_name,home_captain_id,away_captain_id
17,Javi López,Martín Mantovani,90,131
30,Dani García,Martín Mantovani,35,131
194,Fernando Pacheco,Martín Mantovani,53,131
214,Jorge Molina,Martín Mantovani,104,131
229,Álex Granell,Martín Mantovani,194,131
255,Diego Godín,Martín Mantovani,44,131


In [19]:
for i,v in match_results.dtypes.iteritems():
    print(i, v)

match_id int64
date_id int64
venue_id int64
referee_id int64
home_id int64
away_id int64
attendance Int64
match_week int64
xg_home float64
xg_away float64
home_goal int64
away_goal int64
home_captain_name object
away_captain_name object
home_formation object
away_formation object
home_possession float64
away_possession float64
home_shots int64
home_shots_on_target int64
home_distance float64
home_freekicks int64
home_penalty_kicks int64
home_touches int64
home_touches_att_pen int64
home_touches_def_pen int64
home_touches_def_third int64
home_touches_mid_third int64
home_touches_att_third int64
home_touches_live int64
home_dribbles_success int64
home_dribbles_att int64
home_dribbles_players_number int64
home_dribbles_megs int64
home_carries int64
home_carries_tot_dist int64
home_carries_prog_dist int64
home_carries_prog float64
home_carries_one_third float64
home_carries_cpa float64
home_carries_miss int64
home_carries_dis int64
home_receiving_target int64
home_receiving_rec int64
home_

In [20]:
date_dim_df = pd.read_sql("""select 
date_id,
_date_,
_time_,
_day_, 
extract(MONTH from _date_) AS month,
extract(DAY from _date_) AS day_of_month,
extract(DOW from _date_) AS day_of_week,
extract(YEAR from _date_) AS year,
extract(HOUR from _time_) AS hour,
extract(MINUTE from _time_) AS minute
from laliga_wh.match_dates""", con = conn)

In [21]:
date_dim_df

,date_id,_date_,_time_,_day_,month,day_of_month,day_of_week,year,hour,minute
0,1,2017-08-18,20:15:00,Fri,8.0,18.0,5.0,2017.0,20.0,15.0
1,2,2017-08-18,22:15:00,Fri,8.0,18.0,5.0,2017.0,22.0,15.0
2,3,2017-08-19,18:15:00,Sat,8.0,19.0,6.0,2017.0,18.0,15.0
3,4,2017-08-19,20:15:00,Sat,8.0,19.0,6.0,2017.0,20.0,15.0
4,5,2017-08-19,22:15:00,Sat,8.0,19.0,6.0,2017.0,22.0,15.0
...,...,...,...,...,...,...,...,...,...,...
1728,1729,2022-05-20,21:00:00,Fri,5.0,20.0,5.0,2022.0,21.0,0.0
1729,1730,2022-05-21,17:30:00,Sat,5.0,21.0,6.0,2022.0,17.0,30.0
1730,1731,2022-05-22,17:30:00,Sun,5.0,22.0,0.0,2022.0,17.0,30.0
1731,1732,2022-05-22,20:00:00,Sun,5.0,22.0,0.0,2022.0,20.0,0.0


In [22]:
date_dim_df.dtypes

date_id           int64
_date_           object
_time_           object
_day_            object
month           float64
day_of_month    float64
day_of_week     float64
year            float64
hour            float64
minute          float64
dtype: object

In [23]:
date_dim_df['_date_'] = pd.to_datetime(date_dim_df['_date_'])
date_dim_df['month'] = date_dim_df['month'].astype('int')
date_dim_df['day_of_month'] = date_dim_df['day_of_month'].astype('int')
date_dim_df['day_of_week'] = date_dim_df['day_of_week'].astype('int')
date_dim_df['year'] = date_dim_df['year'].astype('int')
date_dim_df['hour'] = date_dim_df['hour'].astype('int')
date_dim_df['minute'] = date_dim_df['minute'].astype('int')

In [24]:
date_dim_df.dtypes

date_id                  int64
_date_          datetime64[ns]
_time_                  object
_day_                   object
month                    int32
day_of_month             int32
day_of_week              int32
year                     int32
hour                     int32
minute                   int32
dtype: object

In [25]:
match_merge = match_results.merge(date_dim_df, on = 'date_id')

In [26]:
match_merge

,match_id,date_id,venue_id,referee_id,home_id,away_id,attendance,match_week,xg_home,xg_away,...,match_result_code,_date_,_time_,_day_,month,day_of_month,day_of_week,year,hour,minute
0,1,1,13,22,17,1,9231,1,1.4,1.2,...,0,2017-08-18,20:15:00,Fri,8,18,5,2017,20,15
1,2,2,25,18,26,16,35971,1,1.9,0.3,...,0,2017-08-18,22:15:00,Fri,8,18,5,2017,22,15
2,3,3,22,7,6,24,16961,1,1.6,2.4,...,1,2017-08-19,18:15:00,Sat,8,19,6,2017,18,15
3,4,4,3,24,12,3,11511,1,2.1,0.8,...,2,2017-08-19,20:15:00,Sat,8,19,6,2017,20,15
4,5,5,18,4,25,10,30487,1,2.1,1.1,...,2,2017-08-19,22:15:00,Sat,8,19,6,2017,22,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,1896,1732,16,4,13,10,17951,38,2.0,0.4,...,2,2022-05-22,20:00:00,Sun,5,22,0,2022,20,0
1896,1897,1732,8,9,21,19,18717,38,0.8,1.3,...,1,2022-05-22,20:00:00,Sun,5,22,0,2022,20,0
1897,1898,1733,1,21,4,28,54850,38,0.7,0.7,...,1,2022-05-22,22:00:00,Sun,5,22,0,2022,22,0
1898,1899,1733,12,18,24,3,23586,38,2.3,0.8,...,1,2022-05-22,22:00:00,Sun,5,22,0,2022,22,0


In [27]:
for i,v in match_merge.dtypes.iteritems():
    print(i, v)

match_id int64
date_id int64
venue_id int64
referee_id int64
home_id int64
away_id int64
attendance Int64
match_week int64
xg_home float64
xg_away float64
home_goal int64
away_goal int64
home_captain_name object
away_captain_name object
home_formation object
away_formation object
home_possession float64
away_possession float64
home_shots int64
home_shots_on_target int64
home_distance float64
home_freekicks int64
home_penalty_kicks int64
home_touches int64
home_touches_att_pen int64
home_touches_def_pen int64
home_touches_def_third int64
home_touches_mid_third int64
home_touches_att_third int64
home_touches_live int64
home_dribbles_success int64
home_dribbles_att int64
home_dribbles_players_number int64
home_dribbles_megs int64
home_carries int64
home_carries_tot_dist int64
home_carries_prog_dist int64
home_carries_prog float64
home_carries_one_third float64
home_carries_cpa float64
home_carries_miss int64
home_carries_dis int64
home_receiving_target int64
home_receiving_rec int64
home_

# Feature Selection